# Bronze Layer –> Raw Banking Data -ApexTrust Bank

Objective:
- Ingest raw banking data as is
- Create Delta tables for audit and traceability
- Preserve original schema (no business rules)
- Enable Time Travel for regulatory audits

#### Read Raw Customers CSV
(PySpark DataFrame API)

What actually happens
- Uses DataFrame API
- Schema inference happens once at read time
- DataFrame can be:
-       -       cached
-       -       reused
-       -       transformed step-by-step
- Logical plan is built programmatically

🔹 Best for
-       1.  ETL pipelines
-       2.  Bronze / Silver / Gold logic
-       3.  Reuse across notebooks
-       4.  Complex transformations

In [0]:
customers_df = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv("/Volumes/apextrust_catalog/landing/operational_data/customers.csv")
)

display(customers_df)


##### Write Customers as Delta (Bronze)

In [0]:
 
customers_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("apextrust_catalog.bronz.customers");

display(customers_df)


#### Read Raw Customers CSV

  (SQL on CSV using csv. data source)


In [0]:
%sql 
SELECT * FROM csv.`/Volumes/apextrust_catalog/landing/operational_data/customers.csv`

What actually happens
- Uses SQL Data Source API
- Schema inference happens every query execution
- File is scanned each time
- Data is treated as external, non-managed

🔹 Best for

-    Quick exploration
-    Ad-hoc analysis
-    One-time validation

| Aspect               | DataFrame API | SQL `csv.`  |
| -------------------- | ------------- | ----------- |
| API Type             | Programmatic  | Declarative |
| Schema inference     | Once          | Every query |
| Reusability          | High          | Low         |
| Caching support      | Easy          | Limited     |
| Pipeline suitability | ✅ Yes         | ❌ No        |
| Performance          | ✅ Better      | ⚠️ Slower   |
| Production use       | ✅ Recommended | ❌ Avoid     |


In [0]:
catalog = 'apextrust_catalog'
schema = 'landing'
volume_name = 'operational_data'
spark.sql(f"CREATE VOLUME IF NOT EXISTS `{catalog}`.`{schema}`.`{volume_name}`")

#### Repeat for Accounts Table


In [0]:
%sql

--we use our catlog 



SELECT current_metastore(), current_catalog(), current_schema();

USE CATALOG apextrust_catalog;
USE SCHEMA bronz;




In [0]:
accounts_df = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv("/Volumes/apextrust_catalog/landing/operational_data/accounts.csv")
)

spark.sql("CREATE SCHEMA IF NOT EXISTS apextrust_catalog.bronz")

accounts_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("apextrust_catalog.bronz.accounts")

display(accounts_df)

#### Repeat for Transactions Table

In [0]:
transactions_df = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv("/Volumes/apextrust_catalog/landing/operational_data/transactions.csv")
)

transactions_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("apextrust_catalog.bronz.transactions")



display(transactions_df)


#### (Added) Branches Table – Dimension Data

In [0]:
branches_df = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv("/Volumes/apextrust_catalog/landing/operational_data/branches.csv")
)

branches_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("apextrust_catalog.bronz.branches")

display(branches_df)

#### Verify Tables (SQL Cell)


In [0]:
%sql

SELECT * FROM apextrust_catalog.bronz.customers;

#### UPDATE & DELETE (Demonstration Only)
    In real projects, Bronze is append-only
    This is for exam understanding

In [0]:
%sql

UPDATE apextrust_catalog.bronz.customers
  SET city = 'chipalun'
  WHERE customer_id = 1;



In [0]:
%sql
SELECT * FROM apextrust_catalog.bronz.customers 
  WHERE customer_id = 1;

In [0]:
%sql 

DELETE FROM apextrust_catalog.bronz.customers
WHERE customer_id = -1;

In [0]:
%sql

SELECT * 
  FROM apextrust_catalog.bronz.customers
  WHERE customer_id = -1

##### Time Travel (Regulatory Audit)

In [0]:
%sql 

DESCRIBE HISTORY apextrust_catalog.bronz.customers;

In [0]:
%sql

SELECT * FROM apextrust_catalog.bronz.customers VERSION AS OF 0;

In [0]:
%sql
SELECT * FROM apextrust_catalog.bronz.customers VERSION AS OF 3;